**Moody's Seasoned Baa Corporate Bond Minus Federal Funds Rate**

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet
!pip install Nasdaq-Data-Link

     |████████████████████████████████| 62.6 MB 120 kB/s 
     |████████████████████████████████| 64 kB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.1 requires cmdstanpy>=1.0.4, but you have cmdstanpy 0.9.5 which is incompatible.
     |████████████████████████████████| 62 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nasdaqdatalink
moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df  = nasdaqdatalink.get("FRED/BAAFF", authtoken="ugpNyJHCrKx_PeswJSpx")

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [4]:
today = datetime.today().strftime('%Y-%m-%d')

moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df.tail()

,Value
Date,
2022-07-11,$3.73
2022-07-12,$3.69
2022-07-13,$3.65
2022-07-14,$3.69
2022-07-15,$3.65


In [5]:
moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9138 entries, 1986-01-02 to 2022-07-15
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   9138 non-null   float64
dtypes: float64(1)
memory usage: 142.8 KB


In [6]:
moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df.isnull().sum()

Value    0
dtype: int64

In [7]:
moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df.reset_index(inplace=True)
moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df.columns

Index(['Date', 'Value'], dtype='object')

In [8]:
df = moodys_seasoned_baa_corporate_bond_minus_federal_funds_rate_df[["Date", "Value"]]

new_names = {
    "Date": "ds", 
    "Value": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
9133,2022-07-11,$3.73
9134,2022-07-12,$3.69
9135,2022-07-13,$3.65
9136,2022-07-14,$3.69
9137,2022-07-15,$3.65


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Moody's Seasoned Baa Corporate Bond Minus Federal Funds Rate",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
9498,2023-07-11
9499,2023-07-12
9500,2023-07-13
9501,2023-07-14
9502,2023-07-15


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
9498,2023-07-11,$2.53,$1.40,$3.70
9499,2023-07-12,$2.52,$1.39,$3.83
9500,2023-07-13,$2.52,$1.34,$3.72
9501,2023-07-14,$2.54,$1.39,$3.68
9502,2023-07-15,$-5.39,$-7.25,$-3.59


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

-5.541281646123344

In [15]:
plot_plotly(m, forecast)

In [16]:
plot_components_plotly(m, forecast)